<a href="https://colab.research.google.com/github/lwa01/229352-STAT-LEARING-FOR-DATA-SCI-2/blob/main/Lab02_Data_Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Statistical Learning for Data Science 2 (229352)
#### Instructor: Donlapark Ponnoprat

#### [Course website](https://donlapark.pages.dev/229352/)

## Lab #2

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve

# For Fashion-MNIST
from tensorflow.keras.datasets import fashion_mnist

# For 20 Newsgroups
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

## Part 1: Marketing Campaign Dataset - Manual Data Preprocessing & Logistic Regression

### Load the Marketing Campaign Dataset ([Data Information](https://archive.ics.uci.edu/dataset/222/bank+marketing))

The data is related with direct marketing campaigns of a Portuguese banking institution. The marketing campaigns were based on phone calls. Often, more than one contact to the same client was required, in order to access if the product (bank term deposit) would be (`'yes'`) or not (`'no'`) subscribed.

In [43]:
bank_url = 'https://raw.githubusercontent.com/donlap/ds352-labs/main/bank.csv'

df = pd.read_csv(bank_url, sep=';', na_values=['unknown'])
df = df.drop(["emp.var.rate", "cons.price.idx", "cons.conf.idx",	"euribor3m", "nr.employed"], axis=1)
print("Shape of the dataset:", df.shape)
df.head()

Shape of the dataset: (41188, 16)


,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,no
1,57,services,married,high.school,NaN,no,no,telephone,may,mon,149,1,999,0,nonexistent,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,no


### Data Exploration

In [28]:
print("--- Missing Values Count ---")
print(df.isnull().sum())

--- Missing Values Count ---
age               0
job             330
marital          80
education      1731
default        8597
housing         990
loan            990
contact           0
month             0
day_of_week       0
duration          0
campaign          0
pdays             0
previous          0
poutcome          0
y                 0
dtype: int64


In [30]:
print("--- Unique Values for Categorical Columns ---")
for col in df.select_dtypes(include='object').columns:
    print(f"\n'{col}' unique values:")
    print(df[col].value_counts(dropna=False)) # Include NaN counts

--- Unique Values for Categorical Columns ---

'job' unique values:
job
admin.           10422
blue-collar       9254
technician        6743
services          3969
management        2924
retired           1720
entrepreneur      1456
self-employed     1421
housemaid         1060
unemployed        1014
student            875
NaN                330
Name: count, dtype: int64

'marital' unique values:
marital
married     24928
single      11568
divorced     4612
NaN            80
Name: count, dtype: int64

'education' unique values:
education
university.degree      12168
high.school             9515
basic.9y                6045
professional.course     5243
basic.4y                4176
basic.6y                2292
NaN                     1731
illiterate                18
Name: count, dtype: int64

'default' unique values:
default
no     32588
NaN     8597
yes        3
Name: count, dtype: int64

'housing' unique values:
housing
yes    21576
no     18622
NaN      990
Name: count, dtype: int64


### Data Preprocessing

In [44]:
# Map target variable 'y' to 0 (no) and 1 (yes)
df['y_new'] = df['y'].map({'no': 0, 'yes': 1})

# Drop 'duration' due to data leakage
# Define features (X) and target (y)
X = df.drop(['y', 'y_new'], axis=1)
y = df['y_new']

# Split the data BEFORE any transformations
# Print data shape
print(X)
print(y)

       age          job  marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
0       56    housemaid  married             basic.4y      no      no   no  telephone   may         mon       261         1    999         0  nonexistent
1       57     services  married          high.school     NaN      no   no  telephone   may         mon       149         1    999         0  nonexistent
2       37     services  married          high.school      no     yes   no  telephone   may         mon       226         1    999         0  nonexistent
3       40       admin.  married             basic.6y      no      no   no  telephone   may         mon       151         1    999         0  nonexistent
4       56     services  married          high.school      no      no  yes  telephone   may         mon       307         1    999         0  nonexistent
...    ...          ...      ...                  ...     ...     ...  ...  

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
X_train, X_test, y_train, y_test

(       age            job   marital            education default housing loan    contact month day_of_week  duration  campaign  pdays  previous     poutcome
 37398   25  self-employed    single    university.degree      no      no   no   cellular   aug         wed       169         2    999         1      failure
 26120   50         admin.   married    university.degree      no     yes  yes   cellular   nov         wed       332         4    999         0  nonexistent
 1265    39    blue-collar   married             basic.6y      no      no   no  telephone   may         thu       124         1    999         0  nonexistent
 1776    60     technician  divorced  professional.course     NaN     yes   no  telephone   may         fri        57         2    999         0  nonexistent
 7446    40         admin.   married          high.school      no      no   no  telephone   may         fri       171         1    999         0  nonexistent
 ...    ...            ...       ...                

We will apply `StandardScaler()`, `OrdinalEncoder()`, and `OneHotEncoder()` on a few selected columns.

**1. Numerical Feature: `age` and `campaign` (Standard Scaling)**

In [48]:
num_cols_demo = ['age', 'campaign']
scaler = StandardScaler()

# Fit the scaler ONLY on the training data
X_train[num_cols_demo] = scaler.fit_transform(X_train[num_cols_demo])
X_test[num_cols_demo] = scaler.transform(X_test[num_cols_demo])

X_test

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome
10417,-0.771787,services,married,high.school,no,no,no,telephone,jun,mon,244,0.156344,999,0,nonexistent
18220,0.567017,services,married,high.school,no,yes,yes,cellular,jul,wed,149,2.669720,999,0,nonexistent
18944,1.332048,blue-collar,married,NaN,no,no,no,cellular,aug,mon,143,-0.561763,999,0,nonexistent
20734,1.427677,technician,married,university.degree,NaN,yes,no,cellular,aug,wed,150,-0.202709,999,0,nonexistent
26983,-0.867416,services,married,basic.6y,no,no,no,cellular,nov,thu,843,0.156344,999,0,nonexistent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28561,-0.484901,technician,married,professional.course,no,yes,no,cellular,apr,wed,251,-0.561763,999,0,nonexistent
29991,2.479594,retired,married,basic.4y,no,yes,no,cellular,apr,tue,63,-0.561763,999,1,failure
30707,0.088872,blue-collar,married,basic.9y,no,yes,no,cellular,may,tue,243,-0.561763,999,2,failure
19787,-0.580530,admin.,married,university.degree,no,yes,no,cellular,aug,fri,193,-0.561763,999,0,nonexistent


In [49]:
X_train.describe()

,age,duration,campaign,pdays,previous
count,2.883100e+04,28831.000000,2.883100e+04,28831.000000,28831.000000
mean,-2.045543e-17,256.906004,-4.682568e-17,962.700219,0.173945
std,1.000017e+00,256.958108,1.000017e+00,186.353384,0.496770
min,-2.206220e+00,0.000000,-5.617632e-01,0.000000,0.000000
25%,-7.717874e-01,103.000000,-5.617632e-01,999.000000,0.000000
50%,-1.980142e-01,179.000000,-2.027095e-01,999.000000,0.000000
75%,6.626456e-01,318.000000,1.563442e-01,999.000000,0.000000
max,5.539718e+00,4918.000000,1.918619e+01,999.000000,7.000000


**2. Ordinal Feature: `education` (Ordinal Encoding with Imputation)**

- **Imputation**

In [50]:
ord_col_demo = ['education']
imputer_ord = SimpleImputer(strategy='most_frequent')
## Write your code here
X_train[ord_col_demo] = imputer_ord.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = imputer_ord.transform(X_test[ord_col_demo])

X_train['education']

,education
37398,university.degree
26120,university.degree
1265,basic.6y
1776,professional.course
7446,high.school
...,...
38485,professional.course
7097,basic.4y
33197,basic.6y
27693,basic.9y


- **Ordinal Encoding**

In [51]:
education_categories = [
    'illiterate', 'basic.4y', 'basic.6y', 'basic.9y', 'high.school',
    'professional.course', 'university.degree', 'masters', 'doctorate'
]

In [53]:
ordinal_encoder = OrdinalEncoder(categories=[education_categories])

## Write your code here
X_train[ord_col_demo] = ordinal_encoder.fit_transform(X_train[ord_col_demo])
X_test[ord_col_demo] = ordinal_encoder.transform(X_test[ord_col_demo])
X_test[ord_col_demo]


,education
10417,4.0
18220,4.0
18944,6.0
20734,6.0
26983,2.0
...,...
28561,5.0
29991,1.0
30707,3.0
19787,6.0


**3. Nominal Feature: `job` (One-Hot Encoding with Imputation)**

- **Imputation**

In [54]:
nom_col_demo = ['job']

imputer_nom = SimpleImputer(strategy='most_frequent')
imputer_nom.fit(X_train[nom_col_demo])

X_train_imputed_nom_demo = imputer_nom.transform(X_train[nom_col_demo])
X_test_imputed_nom_demo = imputer_nom.transform(X_test[nom_col_demo])

- **Nominal Encoding**

In [ ]:
onehot_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
## Write your code here




### **Exercise 1: Apply All Preprocessing & Train Logistic Regression**

Now, it's your turn to apply these preprocessing steps to *all* relevant columns and then train a Logistic Regression model.

**Instructions:**

1.  Look at the Variable Table in [this link](https://archive.ics.uci.edu/dataset/222/bank+marketing).
2. Make lists for `numerical_features`, `ordinal_features`, and `nominal_features`.
3. Preprocess the features. It is safer to make a copy of `X_train` using:
   ```
   X_train_copy = X_train.copy()
   X_test_copy = X_test.copy()
   ```
   and preprocess `X_train_copy` instead.

   **For nominal features, concat the one-hot encoded features using [`pd.concat(..., axis=1)`](https://pandas.pydata.org/docs/reference/api/pandas.concat.html) and drop the old nominal features from the dataframe.**
4. Train Logistic Regression on the preprocessed `X_train_copy` and `y_train`.
5. Evaluate the Model:
    *   Make predictions on the preprocessed `X_test_copy`.
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?


In [ ]:
# --- YOUR CODE FOR EXERCISE 1 STARTS HERE ---




## Part 2: Fashion-MNIST Dataset - Image Classification

### Load Fashion-MNIST Dataset

The Fashion-MNIST dataset consists of 28x28 grayscale images of fashion items.

In [ ]:
(fm_X_train, fm_y_train), (fm_X_test, fm_y_test) = fashion_mnist.load_data()

print(f"Fashion-MNIST Train data shape: {fm_X_train.shape}")
print(f"Fashion-MNIST Train labels shape: {fm_y_train.shape}")
print(f"Fashion-MNIST Test data shape: {fm_X_test.shape}")
print(f"Fashion-MNIST Test labels shape: {fm_y_test.shape}")

In [ ]:
print(f"First image {fm_X_train[0]}")
print(f"First label {fm_y_train[0]}")

### Visualize Fashion-MNIST Images

Let's see what these images look like.

In [ ]:
fashion_mnist_class_names = [
    'T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
    'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'
]

# Visualize the images
## Write your code here



### **Exercise 2: Preprocessing Images (Flatten and Scale)**

Images are 2D arrays (matrices of pixels) and pixel values are integers from 0-255. For Logistic Regression, we need:
*  **Flattening:** Convert each 28x28 image into a 1D array of 784 features.
*  **Scaling:** Normalize pixel values from [0, 255] to [0, 1].

**Instructions:**

1.   **Flatten:** Use the `.reshape()` method (see [documentation](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.reshape.html)). For `fm_X_train_binary` (shape `(num_samples, 28, 28)`), you want to reshape it to `(num_samples, 28*28)`.
2.  **Scale:** Divide the flattened pixel values by 255.0 to get values between 0 and 1.
3.   **Train Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
    *   Fit the model on your *processed* `fm_X_train_scaled` and `fm_y_train`.
4.   **Make Predictions:** Use `predict()` to make predictions on the *processed* `fm_X_test_scaled`.
5.   **Print Classification Report:** Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?
6.   **Visualize Misclassifications:**
    *   Find the indices in `fm_X_test_binary` where your model made incorrect predictions (i.e., `fm_y_pred != fm_y_test`).
    *   Select 5 of these misclassified images.
    *   Plot these images (using `plt.imshow`). For each image, print its true label and its predicted label.

In [ ]:
# --- YOUR CODE FOR EXERCISE 2 STARTS HERE ---





## Part 3: 20 Newsgroups Dataset - Text Classification

### Load 20 Newsgroups Dataset

The 20 newsgroups dataset comprises around 18000 newsgroups posts on 20 topics.

In [ ]:
news_train = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)
news_test = fetch_20newsgroups(subset='test', shuffle=True, random_state=42)

X_train_news, y_train_news = news_train.data, news_train.target
X_test_news, y_test_news = news_test.data, news_test.target

print(f"Number of training documents: {len(X_train_news)}")
print(f"Number of test documents: {len(X_test_news)}")
print(f"Categories: {news_train.target_names}")

### Explore Sample Document

In [ ]:
# Print the first document and its class
## Write your code here



### Preprocessing: Text Vectorization Demonstration with `TfidfVectorizer`

$$
\text{TF-IDF}(t, d, D) = \text{TF}(t, d) \times \text{IDF}(t, D)
$$

Where:

$$
\text{TF}(t, d) = \frac{\text{number of word }t\text{ in } d}{\text{number of words in } d} \quad \text{ and } \quad
\text{IDF}(t, D) = \log\left(\frac{\text{total number of documents}}{\text{number of documents that contain word }t}\right).
$$

In [ ]:
sample_sentences = [
    "This is the first document.",
    "This document is the second document.",
    "And this is the third one.",
    "Is this the first document?"
]

vectorizer = TfidfVectorizer(stop_words='english')

# Fit and transform the sample sentences
sample_vec_output_sparse = # Write your code here

sample_vec_output_dense = sample_vec_output_sparse.toarray()

print(vectorizer.vocabulary_)
print(vectorizer.get_feature_names_out())
print(sample_vec_output_dense)

### **Exercise 3: Apply TF-IDF Vectorization to Full Dataset**

Now, apply `TfidfVectorizer` to the actual training and testing datasets for the 20 Newsgroups classification task.

**Instructions:**

1.  **Initialize `TfidfVectorizer`:**
    *   Initialize `TfidfVectorizer`. Use `stop_words='english'` to remove common words.
2.  **Fit and Transform Training Data:**
    *   Call `fit_transform()` on `X_train_news` to learn the vocabulary and transform the training text into TF-IDF features. Store the result in `X_train_vec`.
3.  **Transform Test Data:**
    *   Call `transform()` on `X_test_news` using the *already fitted* vectorizer. Store the result in `X_test_vec`. **Crucially, do not call `fit_transform()` on the test data!** This would cause data leakage.
4.  **Initialize Logistic Regression:**
    *   Initialize `LogisticRegression(solver='saga')`. `saga` is a good solver when both number of samples and number of features are large.
5.  **Train the Model:**
    *   Fit the model on your `X_train_vec` and `y_train_news`.
6.  **Make Predictions:**
    *   Make predictions using `predict()` on the `X_test_vec`.
7.  **Evaluate the Model:**
    *   Print `classification_report` ([Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html)). What are the accuracy, average precision, average recall, and average f1-score?

In [ ]:
# --- YOUR CODE FOR EXERCISE 3 STARTS HERE ---


